In [10]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [11]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [58]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [63]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)
X

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [20]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [69]:
C = torch.randn((27, 2))


In [25]:
C[5]

tensor([1.1305, 0.3874])

In [28]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([1.1305, 0.3874])

In [34]:
C[X].shape

torch.Size([32, 3, 2])

In [36]:
X[13,2]

tensor(1)

In [38]:
C[X][13,2]

tensor([ 0.7662, -0.3912])

In [40]:
C[1]

tensor([ 0.7662, -0.3912])

In [70]:
emb = C[X]
emb.shape
emb

tensor([[[-1.4208, -0.5003],
         [-1.4208, -0.5003],
         [-1.4208, -0.5003]],

        [[-1.4208, -0.5003],
         [-1.4208, -0.5003],
         [-0.5323,  1.7767]],

        [[-1.4208, -0.5003],
         [-0.5323,  1.7767],
         [-0.1964,  0.1158]],

        [[-0.5323,  1.7767],
         [-0.1964,  0.1158],
         [-0.1964,  0.1158]],

        [[-0.1964,  0.1158],
         [-0.1964,  0.1158],
         [-2.9319, -0.4412]],

        [[-1.4208, -0.5003],
         [-1.4208, -0.5003],
         [-1.4208, -0.5003]],

        [[-1.4208, -0.5003],
         [-1.4208, -0.5003],
         [ 0.2790, -0.2972]],

        [[-1.4208, -0.5003],
         [ 0.2790, -0.2972],
         [ 0.5238,  0.9754]],

        [[ 0.2790, -0.2972],
         [ 0.5238,  0.9754],
         [-0.3725,  0.9437]],

        [[ 0.5238,  0.9754],
         [-0.3725,  0.9437],
         [-1.1131,  0.3025]],

        [[-0.3725,  0.9437],
         [-1.1131,  0.3025],
         [-0.3725,  0.9437]],

        [[-1.1131,  0

In [74]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
# emb @ W1 + b1

In [86]:
F = torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1)

In [85]:
unbinded = torch.unbind(emb, 1)

In [ ]:
# Optimize tensor view manipulation

In [79]:
a = torch.arange(18)

In [83]:
# no memory is being changed/copied/created etc. when .view is being called
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [84]:
# always a 1-dimensional vector
a.storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage._TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [88]:
emb.shape

torch.Size([32, 3, 2])

In [91]:
# more efficient and easier than bind/unbind
emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [97]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
# pytorch can infer missing shape of matrix with -1
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h.shape # 100 activations for each of our 32 examples

torch.Size([32, 100])

In [98]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [102]:
logits = h @ W2 + b2

In [103]:
counts = logits.exp()

In [109]:
prob = counts / counts.sum(1, keepdims=True)

In [114]:
# index into the rows of prob, pluck out the probability assigned to the correct character, and calculate loss
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(12.2272)

In [115]:
# ----------- clearer sequence -----------

In [117]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [119]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [120]:
sum(p.nelement() for p in parameters) # number of parameters in total

11897

In [122]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
counts = logits. exp()
prob = counts / counts.sum(1, keepdims=True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

RuntimeError: mat1 and mat2 shapes cannot be multiplied (160x6 and 30x200)